In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyspark.sql import SparkSession
import time
import matplotlib.pyplot as plt
import pandas as pd

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("Evaluation Performance Delta Lake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Table de résultats
spark.sql("""
CREATE TABLE IF NOT EXISTS delta.`s3a://chu/delta/evaluation_performance` (
    requete STRING,
    configuration STRING,
    temps_execution_s DOUBLE
)
""")

# Fonction de mesure de performance
def mesurer_temps_requete(nom_requete, configuration, requete_sql, nb_repetitions=10):
    temps_exec = []
    for i in range(nb_repetitions):
        t0 = time.time()
        spark.sql(requete_sql).collect()  # Exécution complète
        t1 = time.time()
        temps_exec.append(t1 - t0)
    
    temps_moyen = sum(temps_exec) / len(temps_exec)
    
    spark.sql(f"""
        INSERT INTO delta.`s3a://chu/delta/evaluation_performance`
        VALUES ('{nom_requete}', '{configuration}', {temps_moyen})
    """)
    
    print(f"{nom_requete} ({configuration}) : {temps_moyen:.3f} s (moyenne sur {nb_repetitions} exécutions)")

# -------------------------------------------------------------------
# Exemple 1 : requête sur table non partitionnée
# -------------------------------------------------------------------
requete_non_part = """
SELECT fk_etablissement,
       COUNT(DISTINCT fk_patient) * 100.0 / COUNT(fk_patient) AS taux_consultation
FROM delta.`s3a://chu/delta/fait_consultations`
WHERE fk_etablissement = 'ETAB_X'
    AND date_consult BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY fk_etablissement
"""

mesurer_temps_requete(
    nom_requete="Taux de consultation par établissement en 2020",
    configuration="Non partitionnée",
    requete_sql=requete_non_part
)

# -------------------------------------------------------------------
# Exemple 2 : requête sur table partitionnée
# -------------------------------------------------------------------
requete_part = """
SELECT fk_etablissement,
    COUNT(DISTINCT fk_patient) * 100.0 / COUNT(fk_patient) AS taux_consultation
FROM delta.`s3a://chu/delta/fait_consultations_partitionnee`
WHERE fk_etablissement = 'ETAB_X'
    AND date_consult BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY fk_etablissement
"""

mesurer_temps_requete(
    nom_requete="Taux de consultation par établissement en 2020",
    configuration="Partitionnée",
    requete_sql=requete_part
)

# -------------------------------------------------------------------
# Lecture des résultats et visualisation
# -------------------------------------------------------------------
# Conversion en DataFrame Pandas pour affichage
df_perf = spark.read.format("delta").load("s3a://chu/delta/evaluation_performance").toPandas()

# Graphique de comparaison
plt.figure(figsize=(8, 5))
plt.bar(df_perf["configuration"], df_perf["temps_execution_s"], color=['darkred', 'steelblue'])
plt.title("Comparaison des temps d'exécution (partitionnée vs non partitionnée)")
plt.xlabel("Configuration")
plt.ylabel("Temps d'exécution moyen (s)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()